In [1]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 518.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all th

## LLM

In [2]:
from getpass import getpass

OPENAI_API_KEY = getpass()

import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [3]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives Top 5 Customers based on Sales in current year"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response)

ChatCompletion(id='chatcmpl-8iNWng1wiGwDBCspLHPh4p5AgmJzx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To get the top 5 customers based on sales in the current year, you can use the following SQL query:\n\n```sql\nSELECT u.full_name AS customer_name, SUM(p.price * oi.quantity) AS total_sales\nFROM users u\nJOIN orders o ON u.id = o.user_id\nJOIN order_items oi ON o.id = oi.order_id\nJOIN products p ON oi.product_id = p.id\nWHERE YEAR(o.created_at) = YEAR(CURRENT_DATE())\nGROUP BY u.id\nORDER BY total_sales DESC\nLIMIT 5;\n```\n\nThis query joins the `users`, `orders`, `order_items`, and `products` tables to calculate the total sales for each customer in the current year. It then groups the results by the customer's ID and orders them in descending order based on total sales. Finally, it limits the results to the top 5 customers.", role='assistant', function_call=None, tool_calls=None))], created=1705587565, model='gpt-3.5-turbo-

In [4]:
print(response.choices[0].message.content)

To get the top 5 customers based on sales in the current year, you can use the following SQL query:

```sql
SELECT u.full_name AS customer_name, SUM(p.price * oi.quantity) AS total_sales
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE YEAR(o.created_at) = YEAR(CURRENT_DATE())
GROUP BY u.id
ORDER BY total_sales DESC
LIMIT 5;
```

This query joins the `users`, `orders`, `order_items`, and `products` tables to calculate the total sales for each customer in the current year. It then groups the results by the customer's ID and orders them in descending order based on total sales. Finally, it limits the results to the top 5 customers.


In [5]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List customers where current month sales figures < 50% as of last  month"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To write a SQL query that gives a list of customers where the current month sales figures are less than 50% of the sales figures from the previous month, you can use the following query:

```sql
SELECT u.full_name, u.email
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE o.created_at >= DATE_TRUNC('month', CURRENT_DATE) -- current month
  AND o.created_at < DATE_TRUNC('month', CURRENT_DATE) + INTERVAL '1 month' -- previous month
  AND o.status = 'completed'
GROUP BY u.full_name, u.email
HAVING SUM(p.price * oi.quantity) < 0.5 * (
  SELECT SUM(p.price * oi.quantity)
  FROM users u
  JOIN orders o ON u.id = o.user_id
  JOIN order_items oi ON o.id = oi.order_id
  JOIN products p ON oi.product_id = p.id
  WHERE o.created_at >= DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '1 month' -- previous month
    AND o.created_at < DATE_TRUNC('month', CURRENT_DATE) -- current month
    AND o.status = 'completed'
)
`

In [6]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List products where current week sales > 1.2 times of last week "
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To write a SQL query that gives a list of products where the current week sales are greater than 1.2 times the sales of the previous week, you will need to join the tables and calculate the sales for each week. Here's an example query:

```sql
SELECT p.name, SUM(oi.quantity) AS current_week_sales, SUM(oi_prev.quantity) AS previous_week_sales
FROM products p
JOIN order_items oi ON p.id = oi.product_id
JOIN orders o ON oi.order_id = o.id
JOIN (
    SELECT oi_prev.product_id, SUM(oi_prev.quantity) AS quantity
    FROM order_items oi_prev
    JOIN orders o_prev ON oi_prev.order_id = o_prev.id
    WHERE o_prev.created_at >= DATE_SUB(CURDATE(), INTERVAL 2 WEEK) AND o_prev.created_at < DATE_SUB(CURDATE(), INTERVAL 1 WEEK)
    GROUP BY oi_prev.product_id
) oi_prev ON p.id = oi_prev.product_id
WHERE o.created_at >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) AND o.created_at < CURDATE()
GROUP BY p.id
HAVING SUM(oi.quantity) > 1.2 * SUM(oi_prev.quantity)
```

This query uses subqueries to calculate the

In [7]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List of customers where current month sales figures < 50% as of in the month which is before 3 months the current month"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To get the list of customers where the current month sales figures are less than 50% compared to the sales figures of the month which is three months before the current month, you can use the following SQL query:

```sql
SELECT u.full_name, u.email
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE o.status = 'completed'
  AND DATE_FORMAT(o.created_at, '%Y-%m') = DATE_FORMAT(CURRENT_DATE, '%Y-%m')
  AND p.price * oi.quantity < (
    SELECT 0.5 * SUM(p.price * oi.quantity)
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    JOIN products p ON oi.product_id = p.id
    WHERE o.status = 'completed'
      AND DATE_FORMAT(o.created_at, '%Y-%m') = DATE_FORMAT(DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH), '%Y-%m')
  )
GROUP BY u.id, u.full_name, u.email;
```

This query joins the `users`, `orders`, `order_items`, and `products` tables. It filters the orders based on the status being 'completed' and 

In [8]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Given the following SQL tables, your job is to write queries given a user’s request.\n  Table orders {\n  id int [primary key] \n  user_id int [not null, unique] \n  status varchar \n  created_at varchar \n } \n Table order_items  { \n  order_id int \n  product_id int \n  quantity int \n }\n Table products  { \n  id int [primary key] \n  name varchar \n  merchant_id int [not null] \n  price int \n  status varchar \n  created_at varchar \n  category_id int \n} \n Table users { \n  id int [primary key] \n  full_name varchar \n  email varchar [unique] \n  gender varchar \n  date_of_birth varchar \n  created_at varchar \n  country_code int \n }"
    },
    {
      "role": "user",
      "content": "Write a SQL query which gives List of customers where current month sales figures < 50% as of in the same month of the last year"
    }
  ],
  temperature=0.0,
  top_p=1
)
print(response.choices[0].message.content)

To get the list of customers where the current month sales figures are less than 50% compared to the same month of the last year, you can use the following SQL query:

```sql
SELECT u.full_name, u.email
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id = p.id
WHERE o.status = 'completed'
  AND MONTH(o.created_at) = MONTH(CURRENT_DATE())
  AND YEAR(o.created_at) = YEAR(CURRENT_DATE())
  AND p.price * oi.quantity < (
    SELECT SUM(p.price * oi.quantity)
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    JOIN products p ON oi.product_id = p.id
    WHERE o.status = 'completed'
      AND MONTH(o.created_at) = MONTH(CURRENT_DATE() - INTERVAL 1 YEAR)
      AND YEAR(o.created_at) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR)
  ) * 0.5
GROUP BY u.id, u.full_name, u.email
```

This query joins the `users`, `orders`, `order_items`, and `products` tables. It filters the orders based on the current month and year u